In [3]:
import torch
import pytorch3d.transforms
import numpy as np
import os
import h5py
import numpy as np
from scipy import interpolate
import cv2
from matplotlib import pyplot as plt
from PIL import Image

In [4]:
def data_pairs_adjacent(num_frames):
    # obtain the data_pairs to compute the tarnsfomration between frames and the reference (first) frame
    
    return torch.tensor([[0,n0] for n0 in range(num_frames)])
class LabelTransform():

    def __init__(
        self, 
        label_type, 
        pairs,
        image_points=None,
        tform_image_to_tool=None,
        tform_image_mm_to_tool = None,
        tform_image_pixel_to_mm = None
        ):        
        """
        :param label_type: {"point", "parameter"}
        :param pairs: data pairs, between which the transformations are constructed, returned from pair_samples
        :param image_points: 2-by-num or 3-by-num in pixel, used for computing loss
        :param tform_image_to_tool: transformation from image coordinates to tool coordinates, usually obtained from calibration
        :param tform_image_mm_to_tool: transformation from image coordinates (mm) to tool coordinates
        :param tform_image_pixel_to_mm: transformation from image coordinates (pixel) to image coordinates (mm)
        """

        self.label_type = label_type
        self.pairs = pairs
        self.tform_image_to_tool = tform_image_to_tool
        self.tform_image_mm_to_tool = tform_image_mm_to_tool
        self.tform_image_pixel_to_mm = tform_image_pixel_to_mm
        self.image_points = image_points
        # pre-compute reference points in tool coordinates
        #self.image_points_in_tool= torch.matmul(self.tform_image_to_tool,self.image_points)
        self.tform_tool_to_image = torch.linalg.inv(self.tform_image_to_tool)
        self.tform_tool_to_image_mm = torch.linalg.inv(self.tform_image_mm_to_tool)
        self.tform_image_mm_to_pixel = torch.linalg.inv(self.tform_image_pixel_to_mm) 

        if self.label_type=="point":              
            self.call_function = self.to_points
        
        elif self.label_type=="transform":  
            self.call_function = self.to_transform_t2t   

        elif self.label_type=="parameter":
            self.call_function = self.to_parameters   

        else:
            raise('Unknown label_type!')
    

    def __call__(self, *args, **kwargs):
        return self.call_function(*args, **kwargs)        
    
    def to_points(self, tforms, tforms_inv=None):
        _tforms = self.to_transform_t2t(tforms, tforms_inv)
        return torch.matmul(_tforms, torch.matmul(self.tform_image_pixel_to_mm,self.image_points))[:,:,0:3,:]  # [batch,num_pairs,(x,y,z,1),num_image_points]
 
    def to_transform_t2t(self, tforms, tforms_inv):
        # the label includes the rigid part of calibration matrix, so the transformation is from image(mm) to image(mm), and the final transformed points is in mm, rather in pixel
        # such that the label is Orthogonal Matrix, and then the label could be converted to 6DOF parameter using functions in pytorch3d
        if tforms_inv is None:
            tforms_inv = torch.linalg.inv(tforms)
        
        tforms_world_to_tool0 = tforms_inv[:,self.pairs[:,0],:,:]
        tforms_tool1_to_world = tforms[:,self.pairs[:,1],:,:]
        # tform_tool1_to_tool0 is Orthogonal Matrix
        tform_tool1_to_tool0 = torch.matmul(tforms_world_to_tool0, tforms_tool1_to_world)  # tform_tool1_to_tool0
        # the returned matrix, which is the label, is Orthogonal Matrix
        return torch.matmul(self.tform_tool_to_image_mm[None,None,...],torch.matmul(tform_tool1_to_tool0,self.tform_image_mm_to_tool[None,None,...])) # tform_image1_mm_to_image0_mm

    def to_parameters(self, tforms, tforms_inv):
        _tforms = self.to_transform_t2t(tforms, tforms_inv)
        # only Orthogonal Matrix can be converted to Euler angles
        #Rotation = pytorch3d.transforms.matrix_to_euler_angles(_tforms[:,:,0:3, 0:3], 'ZYX')
        #params = torch.cat((Rotation, _tforms[:,:,0:3, 3]),2)
        return _tforms
def read_calib_matrices(filename_calib):
    # read the calibration matrices from the csv file
    # T{image->tool} = T{image_mm -> tool} * T{image_pix -> image_mm}}
    tform_calib = np.empty((8,4), np.float32)
    with open(filename_calib,'r') as csv_file:
        txt = [i.strip('\n').split(',') for i in csv_file.readlines()]
        tform_calib[0:4,:]=np.array(txt[1:5]).astype(np.float32)
        tform_calib[4:8,:]=np.array(txt[6:10]).astype(np.float32)
    return torch.tensor(tform_calib[0:4,:]),torch.tensor(tform_calib[4:8,:]), torch.tensor(tform_calib[4:8,:] @ tform_calib[0:4,:])


In [ ]:
final_result_gt = np.load('/data/data_ustv/home/ydou8/gasfm/datasets/Euclidean/001_RH_Per_L_DtP_gt.npz')
final_result_gt.files

['K_gt', 'R_gt', 'T_gt', 'M', 'Ps_gt', 'Ns', 'namesList']

In [5]:
filedic = dict(np.load('../datasets/Euclidean/NijoCastleGate.npz',allow_pickle=True))
#gtpath = '/data/data_ustv/home/zzhang679/gasfm/dataset/Euclidean/tus_sp_data/'
#gt_files = [f for f in os.listdir(gtpath) if f.endswith(".npz")]
#gt_files = sorted(gt_files)
# get object names in HDF5 file
gtpath_h5 = '../../TUS_SP/data/frames_transfs/'
gt_files_h5 = [os.path.join(root, name)
             for root, dirs, files in os.walk(gtpath_h5)
             for name in files
             if name.endswith((".h5"))]


In [65]:
for i in range(len(gt_files_h5 )):
    file_without_pre = os.path.basename(gt_files_h5[i]).split('.')
    sub_folder_dir = os.path.split(os.path.dirname(gt_files_h5[i]))[-1]
    test_file_path = os.path.join(gtpath,sub_folder_dir+'_'+file_without_pre[0] + '.npz')
    if os.path.isfile(test_file_path ):
        DataSet = h5py.File(gt_files_h5[i],'r')
        example_transformation = DataSet["tforms"][()]
        DataSet.close()
        test_data  = np.load(test_file_path)
        filedic = dict(np.load('../datasets/Euclidean/NijoCastleGate.npz',allow_pickle=True))
        assert test_data['data1Ps_gt'].shape[0] ==example_transformation.shape[0]
        Ns = np.zeros((test_data['data1Ps_gt'].shape[0],3,3))
        for i in range(Ns.shape[0]):
        #Ns[i,:,:] =  torch.tensor(tform_calib[4:8,:] @ tform_calib[0:4,:])[:3,:3]
            Ns[i,:,:] =  torch.from_numpy(np.eye(3))
        filedic['M'] = test_data['data1M']
        filedic['K_gt'] = Ns
        tform_calib_scale, tform_calib_R_T, tform_calib = read_calib_matrices(os.path.join('../../TUS_SP/data/','calib_matrix.csv'))
        data_pairs_all = data_pairs_adjacent(example_transformation.shape[0])[0:,:]
        transform_label_global_all = LabelTransform(
                    "parameter",
                    pairs=data_pairs_all,
                    image_points=None,
                    tform_image_to_tool=tform_calib,
                    tform_image_mm_to_tool=tform_calib_R_T,
                    tform_image_pixel_to_mm = tform_calib_scale
                    )

        labels_global_parameters = torch.squeeze(transform_label_global_all(torch.from_numpy(example_transformation)[None,...], None))
        Ps_gt = labels_global_parameters[:, :3, :].numpy()
        R_gt = Ns  @ Ps_gt[:, :, :3]
        print(gt_files_h5[i])
        assert np.allclose(R_gt.swapaxes(1, 2) @ R_gt, np.eye(3)[None, :, :],rtol=1e-02,atol=1e-05)    
        filedic['R_gt'] = labels_global_parameters[:,:3,:3].numpy()
        filedic['T_gt'] = labels_global_parameters [:,:3,3].numpy()
        filedic['Ps_gt'] =labels_global_parameters[:, :3, :].numpy()  
        np.savez(os.path.join('../datasets/Euclidean/',sub_folder_dir+'_'+file_without_pre[0] +'_gt.npz'), **filedic)  

../../TUS_SP/data/frames_transfs/010/LH_Par_C_DtP.h5
../../TUS_SP/data/frames_transfs/022/LH_Per_L_DtP.h5
../../TUS_SP/data/frames_transfs/019/RH_Per_C_DtP.h5
../../TUS_SP/data/frames_transfs/028/LH_Par_L_PtD.h5
../../TUS_SP/data/frames_transfs/022/LH_Per_C_DtP.h5
../../TUS_SP/data/frames_transfs/017/LH_Per_C_PtD.h5
../../TUS_SP/data/frames_transfs/019/LH_Par_C_DtP.h5
../../TUS_SP/data/frames_transfs/025/LH_Per_L_PtD.h5
../../TUS_SP/data/frames_transfs/025/RH_Par_L_DtP.h5
../../TUS_SP/data/frames_transfs/015/LH_Par_C_PtD.h5
../../TUS_SP/data/frames_transfs/028/RH_Per_L_DtP.h5
../../TUS_SP/data/frames_transfs/026/RH_Par_S_PtD.h5
../../TUS_SP/data/frames_transfs/024/RH_Par_C_DtP.h5
../../TUS_SP/data/frames_transfs/020/RH_Per_C_PtD.h5
../../TUS_SP/data/frames_transfs/019/RH_Per_L_DtP.h5
../../TUS_SP/data/frames_transfs/017/LH_Per_L_DtP.h5
../../TUS_SP/data/frames_transfs/020/RH_Per_C_DtP.h5
../../TUS_SP/data/frames_transfs/021/RH_Per_S_DtP.h5
../../TUS_SP/data/frames_transfs/021/LH_Par_S_

In [50]:
#/data/data_ustv/home/zzhang679/gasfm/dataset/Euclidean/tus_sp_data/



#filedic['Ps_gt'] = test_data['data1Ps_gt'][:,:3,:]

In [9]:
#Ps_gt = filedic['Ps_gt']
#Ns = np.linalg.inv(filedic['K_gt'])
#N33 = Ns[:, 2, 2][:, None, None]
#Ns /= N33 # Divide by N33 to ensure last row [0, 0, 1] (although generally the case, a small deviation in scale has been observed for e.g. the PantheonParis scene)

In [36]:
#R_gt = Ns  @ Ps_gt[:, :, :3]
#assert np.allclose(R_gt.swapaxes(1, 2) @ R_gt, np.eye(3)[None, :, :],rtol=1e-03,atol=1e-05)
#filedic['R_gt'] = R_gt
#filedic['T_gt'] = None
#gt_files[0].split('_')[0], '_'.join(gt_files[0].split('_')[1:])

('000', 'LH_Par_C_DtP.npz')

In [6]:
#test_data  = np.load('046_RH_Per_L_PtD.npz')
test_data  = np.load('001_RH_Per_L_DtP.npz')
test_data.files

['data1M', 'data1Ns', 'data1Ps_gt']

In [7]:
Ns = np.zeros((test_data['data1Ps_gt'].shape[0],3,3))
for i in range(Ns.shape[0]):
  #Ns[i,:,:] =  torch.tensor(tform_calib[4:8,:] @ tform_calib[0:4,:])[:3,:3]
  Ns[i,:,:] =  torch.from_numpy(np.eye(3))

In [44]:
filedic['M'] = test_data['data1M']
filedic['K_gt'] = Ns

In [45]:
tform_calib_scale, tform_calib_R_T, tform_calib = read_calib_matrices(os.path.join('../../TUS_SP/data/','calib_matrix.csv'))
data_pairs_all = data_pairs_adjacent(test_data['data1Ps_gt'].shape[0])[0:,:]
transform_label_global_all = LabelTransform(
            "parameter",
            pairs=data_pairs_all,
            image_points=None,
            tform_image_to_tool=tform_calib,
            tform_image_mm_to_tool=tform_calib_R_T,
            tform_image_pixel_to_mm = tform_calib_scale
            )

In [46]:
labels_global_parameters = transform_label_global_all(torch.from_numpy(test_data['data1Ps_gt'])[None,...], None).squeeze(0)
labels_global_parameters.shape

torch.Size([511, 4, 4])

In [47]:
#filedic['Ps_gt'] =example_transformation[:,:3,:]
filedic['R_gt'] = labels_global_parameters[:,:3,:3].numpy()
filedic['T_gt'] = labels_global_parameters [:,:3,3].numpy()
filedic['Ps_gt'] =labels_global_parameters[:, :3, :].numpy()

In [48]:
np.savez('001_RH_Per_L_DtP_new.npz', **filedic)

In [83]:
transform_label_global_all(torch.from_numpy(test_data['data1Ps_gt'])[None,...], None)

tensor([[[[ 1.0000e+00, -5.6274e-07, -1.4147e-07, -4.9591e-05],
          [ 4.7456e-08,  1.0000e+00, -1.0241e-07, -1.9836e-04],
          [-2.4849e-08,  1.2231e-06,  1.0000e+00,  1.9455e-04],
          [ 3.0413e-10,  1.5854e-09,  2.8802e-10,  1.0000e+00]],

         [[ 1.0000e+00,  1.5799e-05,  1.0455e-04, -6.1810e-02],
          [-1.6374e-05,  1.0000e+00,  4.8356e-05, -1.0834e-02],
          [-1.0472e-04, -4.7262e-05,  1.0000e+00,  1.5354e-02],
          [ 3.0408e-10,  1.5854e-09,  2.8812e-10,  1.0000e+00]],

         [[ 1.0000e+00, -2.5156e-04,  2.4242e-04, -2.2626e-01],
          [ 2.5089e-04,  1.0000e+00,  5.3151e-04, -8.5831e-02],
          [-2.4271e-04, -5.3036e-04,  1.0000e+00, -5.6770e-02],
          [ 3.0446e-10,  1.5851e-09,  2.8893e-10,  1.0000e+00]],

         ...,

         [[ 9.8282e-01,  7.9313e-02,  1.6669e-01, -1.1418e+02],
          [-5.5851e-02,  9.8843e-01, -1.4102e-01,  2.2680e+01],
          [-1.7595e-01,  1.2928e-01,  9.7587e-01,  2.8040e+01],
          [ 1.5968e

In [22]:
labels_global_parameters.shape

torch.Size([378, 4, 4])

In [8]:
final_result = np.load('/data/data_ustv/home/ydou8/gasfm/results/result_single_multi_4/TRAINING/001_RH_Per_L_DtP_new/predictions/best_predictions.npz')

In [10]:
def get_raw_data(dataset,):
    """
    :return:
    M - Points Matrix (2mxn)
    Ns - Inversed Calibration matrix (Ks-1) (mx3x3)
    Ps_gt - GT projection matrices (mx3x4)
    NBs - Normzlize Bifocal Tensor (En) (3mx3m)
    triplets
    """

    # Init

    # Get bifocal tensors and 2D points
    M = dataset['M']
    Ps_gt = dataset['Ps_gt']
    Ns = np.linalg.inv(dataset['K_gt'])
    N33 = Ns[:, 2, 2][:, None, None]
    Ns /= N33 # Divide by N33 to ensure last row [0, 0, 1] (although generally the case, a small deviation in scale has been observed for e.g. the PantheonParis scene)
    Ps_gt /= np.linalg.det(Ns @ Ps_gt[:, :, :3])[:, None, None]**(1/3) # Likewise, ensure that P is scaled such that P=K*[R  t], where K=inv(N) has final row [0, 0, 1], and R is a rotation
    R_gt = Ns @ Ps_gt[:, :, :3]
    assert np.allclose(R_gt.swapaxes(1, 2) @ R_gt, np.eye(3)[None, :, :],rtol = 1e-02,atol=1e-04)


    M = torch.from_numpy(M).float()
    Ps_gt = torch.from_numpy(Ps_gt).float()
    Ns = torch.from_numpy(Ns).float()

    return M, Ns, Ps_gt

In [42]:
_,_,Ps_gt = get_raw_data(final_result_gt)
Rs = Ps_gt[:,:3,:3]
ts = Ps_gt[:,:3,3]

In [54]:
final_result['Rs'].shape,Rs.shape

((511, 3, 3), torch.Size([511, 3, 3]))

In [55]:
final_result.files

['scene_name',
 'Ks',
 'Ps',
 'Ps_norm',
 'pts3D_pred',
 'pts3D_triangulated',
 'Rs_gt',
 'ts_gt',
 'Rs',
 'ts',
 'cam_centers',
 'cam_centers_gt',
 'Rs_fixed',
 'ts_fixed',
 'pts3D_pred_fixed',
 'pts3D_triangulated_fixed']

In [56]:
def compare_rotations(R1, R2):
    if isinstance(R1, np.ndarray):
        cos_err = (R1 @ np.transpose(R2, [0,2,1])) [:, np.arange(3), np.arange(3)]
        cos_err = (cos_err.sum(axis=-1) - 1) / 2
    else:
        cos_err = (torch.bmm(R1, R2.transpose(1, 2))[:, torch.arange(3), torch.arange(3)].sum(dim=-1) - 1) / 2
    cos_err[cos_err > 1] = 1
    cos_err[cos_err < -1] = -1
    return np.arccos(cos_err) * 180 / np.pi

In [69]:
np.mean(compare_rotations(Rs.numpy(),final_result['Rs']))

4.9799724

In [70]:
np.mean(compare_rotations(Rs.numpy(),final_result['Rs_gt']))

3.4695036

In [43]:
t_gt = -torch.bmm(Rs.inverse(), ts.unsqueeze(-1)).squeeze()
trans = t_gt.mean(dim=0)
scale = (t_gt - trans).norm(p=2, dim=1).mean()

t_gt = (t_gt - trans)/scale
t_gt 

tensor([[ 1.1203, -0.3202,  1.9584],
        [ 1.1214, -0.3203,  1.9619],
        [ 1.1222, -0.3200,  1.9642],
        ...,
        [-0.8587,  0.1901, -1.4409],
        [-0.8576,  0.1958, -1.4431],
        [-0.8589,  0.1988, -1.4411]])

In [38]:
Vs_invT = torch.from_numpy(final_result["Ps_norm"][:, 0:3, 0:3])
Vs = torch.inverse(Vs_invT).transpose(1, 2)
ts_ = torch.bmm(-Vs.transpose(1, 2), torch.from_numpy(final_result["Ps"][:, 0:3, 3]).unsqueeze(dim=-1)).squeeze()

In [40]:
ts_,t_gt,ts

(tensor([[ 0.4845, -0.1264,  1.2803],
         [ 0.4794, -0.1237,  1.2672],
         [ 0.4980, -0.1282,  1.3035],
         ...,
         [-0.3262,  0.0581, -0.7350],
         [-0.3530,  0.0742, -0.8380],
         [-0.2914,  0.0466, -0.6126]]),
 tensor([[ 1.1203, -0.3202,  1.9584],
         [ 1.1214, -0.3203,  1.9619],
         [ 1.1222, -0.3200,  1.9642],
         ...,
         [-0.8587,  0.1901, -1.4409],
         [-0.8576,  0.1958, -1.4431],
         [-0.8589,  0.1988, -1.4411]]),
 tensor([[ 0.0000e+00, -1.8311e-04,  1.5640e-04],
         [-5.1903e-02,  1.7014e-03, -1.6436e-01],
         [-8.7204e-02, -9.1934e-03, -2.7888e-01],
         ...,
         [ 1.2111e+02, -2.7082e+01,  1.4315e+02],
         [ 1.2112e+02, -2.7208e+01,  1.4324e+02],
         [ 1.2112e+02, -2.7240e+01,  1.4320e+02]]))

In [63]:
translation_err = (t_gt - ts_).norm(p=2, dim=1)
translation_err.mean()

tensor(0.5351)

In [26]:
Vs_invT = torch.from_numpy(final_result["Rs_gt"])
Vs = torch.inverse(Vs_invT).transpose(1, 2)
ts_ = torch.bmm(-Vs.transpose(1, 2), torch.from_numpy(final_result["ts_gt"]).unsqueeze(dim=-1)).squeeze()

In [44]:
translation_err = (ts -final_result["ts_gt"]).norm(p=2, dim=1)
translation_err.mean()

tensor(188.0374)

In [17]:
orient_err = (Rs_pred_orth -Rs_gt_orth  ).norm(p=2, dim=1)
orient_err.mean() 

NameError: name 'Rs_pred_orth' is not defined

In [84]:
(torch.from_numpy(final_result['ts_gt']) - torch.from_numpy(final_result['ts'])).norm(p=2, dim=1).mean()

tensor(66.3591)

In [85]:
final_result['ts_gt']

array([[ 3.81469945e-05,  3.81469399e-05, -3.43322863e-05],
       [-1.76057257e-02,  9.58820339e-03,  2.55114101e-02],
       [ 2.28136610e-02,  3.12431413e-03,  1.18099250e-01],
       ...,
       [ 4.54642715e+01, -1.32920036e+01,  1.20526131e+02],
       [ 4.56145401e+01, -1.33051891e+01,  1.20694695e+02],
       [ 4.58041458e+01, -1.33368311e+01,  1.20908791e+02]], dtype=float32)

In [86]:
final_result['ts']

array([[-0.5931242 ,  0.23268665, -1.9114772 ],
       [-0.5957601 ,  0.23313114, -1.9251986 ],
       [-0.5962507 ,  0.23343615, -1.916431  ],
       ...,
       [ 0.8368802 , -0.18274483,  1.8409928 ],
       [ 0.8430807 , -0.18310413,  1.8530046 ],
       [ 0.742508  , -0.17565493,  1.6709515 ]], dtype=float32)